[View in Colaboratory](https://colab.research.google.com/github/brucecmd/learn_gluon/blob/master/gluon_practice_create_mlp.ipynb)

In [0]:
from mxnet.gluon import nn
from mxnet import nd

In [0]:
# 可以利用继承Block的方式，自己定义一个MLP方法，而不用是Sequential的那个
class MLP(nn.Block):
    pass
    def __init__(self, **kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')
        self.output = nn.Dense(10)
    
    def forward(self, x):
        return self.output(self.hidden(x))

In [0]:
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet import gluon,autograd

In [0]:
mnist_train = gdata.vision.FashionMNIST(train=True)
mnist_test = gdata.vision.FashionMNIST(train=False)

In [0]:
batch_size = 256
transformer = gdata.vision.transforms.ToTensor()
train_iter = gdata.DataLoader(mnist_train.transform_first(transformer), batch_size, shuffle=True)
test_iter = gdata.DataLoader(mnist_test.transform_first(transformer), batch_size, shuffle=True)

In [0]:
net = MLP()
net.initialize()

In [0]:
loss_func = gloss.SoftmaxCrossEntropyLoss()

In [0]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate':0.1})

In [0]:
def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1)==y.astype('float32')).mean().asscalar()
  
def estimate_accuracy(data_iter, net):
    total_acc = 0
    for data, label in data_iter:
        y_hat = net(data)
        acc = accuracy(y_hat, label)
        total_acc += acc
    return total_acc / len(data_iter)

In [155]:
epochs = 10
batch_size = 256
for i in range(epochs):
    for data, label in train_iter:
        with autograd.record():
            y_hat = net(data)
            l = loss_func(y_hat, label)
        l.backward()
        trainer.step(batch_size)
    train_acc = estimate_accuracy(train_iter, net)
    test_acc = estimate_accuracy(test_iter, net)
    print('epoch[%d], train acc[%f], test acc[%f]'%(i, train_acc, test_acc))

epoch[0], train acc[0.805358], test acc[0.802637]
epoch[1], train acc[0.829915], test acc[0.831836]
epoch[2], train acc[0.841146], test acc[0.844238]
epoch[3], train acc[0.849983], test acc[0.848242]
epoch[4], train acc[0.855785], test acc[0.855469]
epoch[5], train acc[0.856970], test acc[0.855078]
epoch[6], train acc[0.863641], test acc[0.859082]
epoch[7], train acc[0.868096], test acc[0.862891]
epoch[8], train acc[0.874485], test acc[0.871875]
epoch[9], train acc[0.873914], test acc[0.866016]
